# PeMS08 data prediction using LSTM Graph Laplacian Regularized Tensor Factorization

> About the author: Jinming Yang (yangjm67@sjtu.edu.cn), Center for Intelligent Transportation Systems and Unmanned Aerial Systems Applications Research, School of Naval Architecture, Ocean and Civil Engineering, Shanghai Jiao Tong University, Shanghai 200240, China. 

## Data Organization: Tensor Structure

We consider a high dimensional dataset collected from $M$ sensors for $T$ time steps. Each sensor collects $N$ different kinds of data. We express spatio-temporal dataset as a tensor $\mathcal{Y}\in\mathbb{R}^{M\times N\times T}$. The three way tensor $\mathcal{Y}$ can be factorized into three low rank feature matrices: spatial feature matrix $\boldsymbol{U}\in \mathbb{R}^{M\times r}$, data type feature matrix $\boldsymbol{V}\in \mathbb{R}^{N\times r}$ and temporal feature matrix $\boldsymbol{X}\in \mathbb{R}^{T\times r}$.


## Long-short Term Memory Regularized Tensor Factorization(LSTM-ReTF)

LSTM Regularized Tensor Factorization (LSTM-ReTF) is an approach to incorporate temporal dependencies into tensor CP factorization model which utilizes the well-studied Long-short term memory(LSTM) neural network to model temporal dependencies among temporal feature vectors ${\boldsymbol{x}_t}$ explicitly. Let $f()$ stands for the feed forward process of the LSTM network temporal regularizer, then the temporal dependencies can be described as follows:

$$
\boldsymbol{x}'_t = f(\boldsymbol{x}_{t-l_1},\boldsymbol{x}_{t-l_2},...,\boldsymbol{x}_{t-l_d})
$$

Where the lag set $\mathcal{L}=\left\{l_1,l_2,...,l_d\right\}$ (e.g., $\mathcal{L}=\left\{1,2,...,24\right\}$) indicates  the temporal correlation topology. We further define the LSTM temporal regularizer as follows:

$$\mathcal{R}_{t}\left(X, var(f)\right) = \frac{1}{2}\sum_{t=l_d}^n\biggl(x_t - f(x_{t-l_1},x_{t-l_2},...,x_{t-l_d})\biggr)^2$$

where, $var(f)$ stands for the coefficient set in LSTM network, and $f()$ is the feed forward process of the LSTM network.

Thus, LSTM-ReTF is given by solving
$$
\min_{U,V,X,\text{var}(f)}~~\sum_{(i,j,t)\in\Omega}\left(y_{ijt}-\sum_{r=1}^{R}u_{ir}v_{jr}x_{tr}\right)^2
+\frac{\lambda_{w}\eta}{2}\sum_{i=1}^{M}\left\|\boldsymbol{w}_{i}\right\|_{2}^{2}+\frac{\lambda_{v}\eta}{2}\sum_{k=1}^{N}\left\|\boldsymbol{v}_{k}\right\|_{2}^{2}+\frac{\lambda_{x}\eta}{2}\sum_{t=1}^{T}\left\|\boldsymbol{x}_{t}\right\|_{2}^{2} \\
+\frac{\lambda_{w}}{2}\underbrace{\sum_{i,j}E_{i,j}^w(\boldsymbol{w}_i - \boldsymbol{w}_j)^2}_{\text{spatial regularizer}}+\underbrace{\frac{\lambda_x}{2}\sum_{t=l_d+1}^n\biggl(\boldsymbol{x}_t - \boldsymbol{x}'_t\biggr)^2}_{\text{LSTM network temporal regularizer}}
$$

where $E_{i,j}^w$ is the edge weight of sensor $i$ and $j$ in the spatial topology graph.

### Solving the above minimization problem using alternative least square method(ALS)

Supper script denotes the iteration times.

#### Updates for spatial feature vectors $\boldsymbol{w}_i, i = 1, 2, ..., M$
$$
\boldsymbol{w}_i^{(p+1)} = \left(\sum_{j,t:(i,j,t)\in\Omega}\left(\boldsymbol{v}_{j}^{(p)}\odot\boldsymbol{x}_{t}^{(p)}\right)\left(\boldsymbol{v}_{j}^{(p)}\odot\boldsymbol{x}_{t}^{(p)}\right)^{\top}+\lambda_{u}\boldsymbol{I} + \lambda_u\sum_{k}E_{i,k}^u\boldsymbol{I} \right)^{-1} \left(\sum_{j,t:(i,j,t)\in\Omega}\left(\boldsymbol{v}_{j}^{(p)}\odot\boldsymbol{x}_{t}^{(p)}\right)y_{ijt} + \lambda_u\sum_{k}E_{i,k}^u \boldsymbol{u}_k^{(p)}\right)
$$

#### Updates for data type feature vectors $\boldsymbol{v}_k, k = 1, 2, ..., N$
$$
\boldsymbol{v}_{k}^{(p+1)} = \left(\sum_{i,t:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{x}_{t}^{(p)}\right)\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{x}_{t}^{(p)}\right)^{\top}+\lambda_{v}I\right)^{-1}\sum_{i,t:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{x}_{t}^{(p)}\right)y_{ikt}
$$

#### Updates for temporal feature vectors $\boldsymbol{x}_t, t = 1, 2, ..., l_d$
$$
\boldsymbol{x}_{t}^{(p+1)} = \biggl(\sum_{i,k:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)^{\top}+\lambda_x\eta I \biggr)^{-1} \biggl(\sum_{i,k:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)y_{ikt}\biggr)
$$

#### Updates for temporal vectors $\boldsymbol{x}_t, t = l_d + 1, l_d + 2, ..., T$
$$
\boldsymbol{x}_{t}^{(p+1)} = \biggl(\sum_{i,k:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)^{\top} + \lambda_x\eta I+\lambda_x I \biggr)^{-1} \biggl(\sum_{i,k:(i,k,t)\in\Omega}\left(\boldsymbol{w}_{i}^{(p+1)}\odot\boldsymbol{v}_{k}^{(p+1)}\right)y_{ikt} + \lambda_x {{\boldsymbol{x}'_t}^{(p+1)}} \biggr)
$$

where $\odot$ denotes element-wise product and ${{\boldsymbol{x}'_t}^{(p+1)}} = f(\boldsymbol{x}_{t-l_1}^{(p+1)},\boldsymbol{x}_{t-l_2}^{(p+1)},...,\boldsymbol{x}_{t-l_d}^{(p+1)}| \theta^{(p)})$.

In [1]:
import numpy as np
import scipy.io
from numpy.linalg import inv as inv
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
import time

Using TensorFlow backend.
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

## Some Computation Concepts

### Kronecker product

- **Definition**:

Given two matrices $A\in\mathbb{R}^{m_1\times n_1}$ and $B\in\mathbb{R}^{m_2\times n_2}$, then, the **Kronecker product** between these two matrices is defined as

$$A\otimes B=\left[ \begin{array}{cccc} a_{11}B & a_{12}B & \cdots & a_{1m_2}B \\ a_{21}B & a_{22}B & \cdots & a_{2m_2}B \\ \vdots & \vdots & \ddots & \vdots \\ a_{m_11}B & a_{m_12}B & \cdots & a_{m_1m_2}B \\ \end{array} \right]$$
where the symbol $\otimes$ denotes Kronecker product, and the size of resulted $A\otimes B$ is $(m_1m_2)\times (n_1n_2)$ (i.e., $m_1\times m_2$ columns and $n_1\times n_2$ rows).

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]$ and $B=\left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10 \\ \end{array} \right]$, then, we have

$$A\otimes B=\left[ \begin{array}{cc} 1\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 2\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ 3\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 4\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cccccc} 5 & 6 & 7 & 10 & 12 & 14 \\ 8 & 9 & 10 & 16 & 18 & 20 \\ 15 & 18 & 21 & 20 & 24 & 28 \\ 24 & 27 & 30 & 32 & 36 & 40 \\ \end{array} \right]\in\mathbb{R}^{4\times 6}.$$

### Khatri-Rao product (`kr_prod`)

- **Definition**:

Given two matrices $A=\left( \boldsymbol{a}_1,\boldsymbol{a}_2,...,\boldsymbol{a}_r \right)\in\mathbb{R}^{m\times r}$ and $B=\left( \boldsymbol{b}_1,\boldsymbol{b}_2,...,\boldsymbol{b}_r \right)\in\mathbb{R}^{n\times r}$ with same number of columns, then, the **Khatri-Rao product** (or **column-wise Kronecker product**) between $A$ and $B$ is given as follows,

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2,...,\boldsymbol{a}_r\otimes \boldsymbol{b}_r \right)\in\mathbb{R}^{(mn)\times r}$$
where the symbol $\odot$ denotes Khatri-Rao product, and $\otimes$ denotes Kronecker product.

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]=\left( \boldsymbol{a}_1,\boldsymbol{a}_2 \right) $ and $B=\left[ \begin{array}{cc} 5 & 6 \\ 7 & 8 \\ 9 & 10 \\ \end{array} \right]=\left( \boldsymbol{b}_1,\boldsymbol{b}_2 \right) $, then, we have

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2 \right) $$

$$=\left[ \begin{array}{cc} \left[ \begin{array}{c} 1 \\ 3 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 5 \\ 7 \\ 9 \\ \end{array} \right] & \left[ \begin{array}{c} 2 \\ 4 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 6 \\ 8 \\ 10 \\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cc} 5 & 12 \\ 7 & 16 \\ 9 & 20 \\ 15 & 24 \\ 21 & 32 \\ 27 & 40 \\ \end{array} \right]\in\mathbb{R}^{6\times 2}.$$

In [2]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

In [3]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8], [9, 10]])
print(kr_prod(A, B))

[[ 5 12]
 [ 7 16]
 [ 9 20]
 [15 24]
 [21 32]
 [27 40]]


### Function to combine feature matrices to data tensor

In [4]:
def cp_combine(U, V, X):
    return np.einsum('is, js, ts -> ijt', U, V, X)

### Function to unfold tensor to matrix along specific dimension

In [5]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

### Load the PeMSD8 dataset

In [76]:
directory = '../Datasets/PEMS08/'

missing_rate = 0.2
mode = 'PM'

Dist = np.load(directory + 'Dist.npy')
A = np.load(directory + 'Adj.npy')
dense_tensor = np.load( directory + 'PeMS08.npy')

dim1, dim2, dim3 = dense_tensor.shape
# dense_mat = dense_tensor.reshape(dim1 * dim2, dim3)
# =============================================================================
### Point-wise missing (PM) scenario
### Set the PM scenario by:
if mode == 'PM':
    pm_random_mat = np.load(directory + 'pm_random_mat.npy')
    binary_mat = np.round(pm_random_mat + 0.5 - missing_rate)
    sparse_tensor = dense_tensor.copy()
    for i in range(dim2):
        sparse_tensor[:, i, :] = dense_tensor[:, i, :] * binary_mat
# =============================================================================
# =============================================================================
### Continuous-random missing (CM) scenario
### Set the CM scenario by:
if mode == 'CM':
    cm_random_mat = np.load(directory + 'cm_random_mat.npy')
    binary_mat = np.round(cm_random_mat + 0.5 - missing_rate)
    dense_tensor_reshape = dense_tensor.reshape(dim1, dim2, 62, 288)
    sparse_tensor_reshape = dense_tensor_reshape.copy()
    for i in range(dim1):
        for j in range(62):
            sparse_tensor_reshape[i, :, j, :] = dense_tensor_reshape[i, :, j, :] * binary_mat[i, j]
    sparse_tensor = sparse_tensor_reshape.reshape(dim1, dim2, dim3)
# =============================================================================
print('Missing rate = %s %.1f'%(mode, missing_rate))

Missing rate = PM 0.2


## Scaled inverse distance weighting

In [78]:
def graph_weight_cal(dist, epsilon = 0.5):
    dim = dist.shape[0]
    distances = dist[np.nonzero(dist)].flatten()
    std = distances.std()
    print(std)
    std_square = 5 * std ** 2
    A = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            if dist[i][j] > 0:
                weight = np.exp(- dist[i][j] ** 2 / std_square)
                if i != j and weight >= epsilon:
                    A[i][j] = weight
    return A
A = graph_weight_cal(Dist, epsilon = 0.5)

217.6933921392941


In [79]:
A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.74723109, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.74723109, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.9392094 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.9392094 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Problem setting
This work mainly focuses on online spatial temporal data prediction and imputation, which is utilizing currently observed data(maybe incomplete) and history data(maybe incomplete) to make prediction of future data. In the mean time, the newly oberved incomplete data can be repaired.

In [80]:
sparse_tensor = sparse_tensor[:, :, 8928:]
dense_tensor = dense_tensor[:, :, 8928:]
dim1, dim2, dim3 = sparse_tensor.shape

### Creating training set and test set

In [81]:
test_len = 2880
train_len = dim3 - test_len
training_set = sparse_tensor[:, :, :train_len]
test_set = sparse_tensor[:, :, train_len:]
print('The size of training set is:')
print(training_set.shape)
print()
print('The size of test set is:')
print(test_set.shape)

The size of training set is:
(170, 3, 6048)

The size of test set is:
(170, 3, 2880)


### Saving ground truth(real value)

In [82]:
training_ground_truth = dense_tensor[:, :, :train_len]
test_ground_truth = dense_tensor[:, :, train_len:]
print('The size of training set ground truth is:')
print(training_ground_truth.shape)
print()
print('The size of test set ground truth is:')
print(test_ground_truth.shape)

The size of training set ground truth is:
(170, 3, 6048)

The size of test set ground truth is:
(170, 3, 2880)


## Train LSTM-GL-ReTF spatial temporal feature matrices and LSTM coefficients

Bofore moving to the online prediction part of the framework, static data features(spatial feature matrix `W`, data type feature matrix `V` and temporal feature matrix `X`) and LSTM network coefficients(`var(f)`) should be trained first.

The following function is used to generate training samples for the LSTM neural network:

- `dataset` is the spatial temporal matrix(training data matrix).
- `rate` ranging from $(0, 1]$ stands for the sampling rate.
- `time_lags` stands for the leg set which denotes the temporal correlation topology.

In [83]:
def create_lstm_samples(dataset, time_lags, rate):
    dataX, dataY = [], []
    data_len = dataset.shape[0] - np.max(time_lags)
    t_sample = np.random.choice(data_len, int(rate * data_len), replace = False)
    
    for t in t_sample:
        a = dataset[t + np.max(time_lags) - time_lags, :][::-1]
        dataX.append(a)
        dataY.append(dataset[t + np.max(time_lags), :])
    return np.array(dataX), np.array(dataY)

The following function creates a LSTM network temporal regularizer. The input layer of the network has `rank` number of units, the LSTM layer has `rank` number of units and the full connection layer also has `rank` number of units.

In [84]:
def lstmmodel(rank, lag_len):
    # create the LSTM network
    model = Sequential()
#     model.add(LSTM(rank, input_shape = (lag_len, rank), return_sequences = True)) # If you need multi-layer LSTM
    model.add(LSTM(rank, input_shape = (lag_len, rank)))
    model.add(Dense(rank))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

## Error calculator
<div class="alert alert-block alert-warning">
<ul>
<li><b><code>mean_absolute_percentage_error</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>root_mean_squared_error</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-{y}'_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and ${y}'_i$ are the actual value and its estimation, respectively.

In [85]:
def mean_absolute_percentage_error(y_true, y_pred, pos): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true[pos] - y_pred[pos]) / y_true[pos])) * 100
def root_mean_squared_error(y_true, y_pred, pos): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(np.mean(np.square(y_true[pos] - y_pred[pos])))

### LSTM-ReTF training algorithm

The function **LSTM_ReTF** is used to train spatial temporal feature matrices and LSTM network parameters.

- `sparse_tensor` is the training set spatial temporal tensor.
- `invD` is the pre-calculated scaled inverse distance matrix.
- `init` is the initiated hyperparameters of LSTM-ReMF which includes the initiated spatial matrix `W` and the initiated temporal matrix `X`.
- `time_lags` stands for the leg set which denotes the temporal correlation topology.
- `lambda_w`, `lambda_x` and `eta` are regularizer parameters. 
- `sampling rate` is the ratio of data used to train the LSTM-full connection network.
- `maxiter` is the maxiter time.
- `track` is a 0 or 1 parameter that indicates whether to compute errors while training.
- `patience` is the tolerance waiting step number. It is only required when `track` variable is 1.
- `dense_tensor` is the training ground truth without data missing simulation. It is only required when `track` variable is 1.

In [86]:
def LSTM_GL_ReTF(sparse_tensor, invD, init, time_lags, lambda_w, lambda_g, lambda_v, lambda_x, eta, sampling_rate, maxiter, track, dense_tensor = 0):
    W = init["W"]
    V = init["V"]
    X = init["X"]
    dim1, dim2, dim3 = sparse_tensor.shape
    binary_tensor = np.zeros((dim1, dim2, dim3))
    position = np.where((sparse_tensor != 0))
    binary_tensor[position] = 1
    d = len(time_lags)
    max_lags = np.max(time_lags)
    r = X.shape[1]
    if track:
        pos = np.where((sparse_tensor == 0) & (dense_tensor != 0))
    model = lstmmodel(r, d)
    model_reverse = lstmmodel(r, d)
    start_time = time.time()

    for iters in range(maxiter):
        var1 = kr_prod(X, V).T #(r, n*t)
        var2 = kr_prod(var1, var1) # (r*r, n*t) 
        mat1 = ten2mat(binary_tensor, 0).T # (n*t, m)
        mat2 = ten2mat(sparse_tensor, 0).T # (n*t, m)
        for i in range(dim1):
            vec1 = np.matmul(W.T, invD[i, :])
            var_Lambda1 = np.matmul(var2, mat1[:, i]).reshape([rank, rank]) + lambda_w * np.eye(rank) + lambda_g * np.eye(rank) * np.sum(invD[i, :])
            inv_var_Lambda1 = np.linalg.inv((var_Lambda1 + var_Lambda1.T)/2)
            W[i, :] = np.matmul(inv_var_Lambda1, np.matmul(var1, mat2[:, i]) + lambda_g * vec1)

        var1 = kr_prod(X, W).T
        var2 = kr_prod(var1, var1)
        mat1 = ten2mat(binary_tensor, 1).T # (m*t, n)
        mat2 = ten2mat(sparse_tensor, 1).T # (m*t, n)
        for j in range(dim2):
            var_Lambda1 = np.matmul(var2, mat1[:, j]).reshape([rank, rank]) + lambda_v * eta * np.eye(rank)
            inv_var_Lambda1 = np.linalg.inv((var_Lambda1 + var_Lambda1.T)/2)
            V[j, :] = np.matmul(inv_var_Lambda1, np.matmul(var1, mat2[:, j]))
        
        var1 = kr_prod(V, W).T # (r, m*n)
        var2 = kr_prod(var1, var1) # (r*r, m*n)
        mat1 = ten2mat(binary_tensor, 2).T # (m*n, n)
        mat2 = ten2mat(sparse_tensor, 2).T # (m*n, n)
        for t in range(dim3):
            if iters == 0 or t < max_lags:
                var_Lambda1 = np.matmul(var2, mat1[:, t]).reshape([rank, rank]) + lambda_x * eta * np.eye(rank)
                X[t, :] = np.matmul(inv((var_Lambda1 + var_Lambda1.T)/2), np.matmul(var1, mat2[:, t]))
            else:
                var_Lambda1 = np.matmul(var2, mat1[:, t]).reshape([rank, rank]) + lambda_x * np.eye(rank) + lambda_x * eta * np.eye(rank)
                X_hat = X[t - time_lags, :][::-1]
                X_hat_feed = X_hat[np.newaxis, :, :]
                Qt =  model.predict(X_hat_feed)[0]
                X[t, :] = np.matmul(inv((var_Lambda1 + var_Lambda1.T)/2),
                                       (np.matmul(var1, mat2[:, t]) + lambda_x * Qt))

        if iters == 0:
            lstmX, lstmY = create_lstm_samples(X, time_lags, 1)
            model.fit(lstmX, lstmY, epochs=20, batch_size=50, verbose=0)
        else:
            lstmX, lstmY = create_lstm_samples(X, time_lags, sampling_rate)
            model.fit(lstmX, lstmY, epochs=2, batch_size=10, verbose=0)
        if (iters + 1) % 10 == 0:
            print('Iterations: %d, time cost: %ds'%((iters + 1), (time.time() - start_time)))
            start_time = time.time()
            if track:
                tensor_hat = cp_combine(W, V, X)
                tensor_hat[position] = sparse_tensor[position]
                tensor_hat[tensor_hat < 0] = 0
                rmse = root_mean_squared_error(dense_tensor, tensor_hat, pos)
                mape = mean_absolute_percentage_error(dense_tensor, tensor_hat, pos)
                print(np.mean(W))
                print(np.mean(V))
                print(np.mean(X))
                print('Imputation RMSE = %.2f'%rmse)
                print('Imputation MAPE = %.2f'%mape)
            print()
#     model.save('model_save\lstm_trained1.h5')
    tensor_hat = cp_combine(W, V, X)
    tensor_hat[position] = sparse_tensor[position]
    tensor_hat[tensor_hat < 0] = 0
    return tensor_hat, W, V, X, model

### Training process

In [87]:
training_set.shape

(170, 3, 6048)

In [88]:
np.random.seed(seed=10)
rank = 20
maxiter = 100
eta = 0.1
lambda_w = 100
lambda_g = 100
lambda_v = 100
lambda_x = 100
sampling_rate = 1.0
time_lags = np.array([1, 2, 288])#np.arange(1, 25, 1)
track = True
dim1, dim2, dim3 = training_set.shape
init = {"W": 0.1 * np.random.rand(dim1, rank), "V": 0.1 * np.random.rand(dim2, rank), "X": 0.1 * np.random.rand(dim3, rank)}
tensor_hat, W, V, X, model = LSTM_GL_ReTF(training_set, A, init, time_lags, lambda_w, lambda_g, lambda_v,
                                       lambda_x, eta, sampling_rate, maxiter, track, training_ground_truth)

Iterations: 10, time cost: 153s
9.22160435061816
5.013191177216398
-0.30011181931667424
Imputation RMSE = 17.25
Imputation MAPE = 22.64

Iterations: 20, time cost: 135s
8.028425640190587
4.956336391614907
-0.22263339333204182
Imputation RMSE = 17.67
Imputation MAPE = 22.12

Iterations: 30, time cost: 137s
7.232722486806132
4.943137785924223
-0.17490110004565443
Imputation RMSE = 17.94
Imputation MAPE = 21.83

Iterations: 40, time cost: 133s
6.532752259708474
4.935415398521771
-0.14243826217865463
Imputation RMSE = 18.13
Imputation MAPE = 21.63

Iterations: 50, time cost: 133s
5.909669403189389
4.930665727301137
-0.11550427206491597
Imputation RMSE = 18.26
Imputation MAPE = 21.46

Iterations: 60, time cost: 143s
5.358603390949798
4.928666589760706
-0.09318869167351733
Imputation RMSE = 18.37
Imputation MAPE = 21.31

Iterations: 70, time cost: 144s
4.869812211397462
4.929795001203834
-0.07400434998983998
Imputation RMSE = 18.45
Imputation MAPE = 21.23

Iterations: 80, time cost: 145s
4.4

## Online prediction and imputation with Online LSTM-ReTF
In the context of spatiotemporal data online prediction and imputation, LSTM-ReTF takes current observation matrix $\boldsymbol{Y}_{t}\in\mathbb{R}^{M\times N}$ to update the previously predicted temporal feature vector $\boldsymbol{x}'_t$ and impute possible missing entries in real time observation $\boldsymbol{Y}_{t}$. Then, LSTM-ReTF use history temporal feature vectors $\boldsymbol{x}_{t +1 - l}, l\in \mathcal{L}$ to make forecast of future temporal feature vecotr $x_{t+1}$. Finally, forecasted temporal feature vector $\boldsymbol{x}_{t+1}$ will be multiplied by spatial embedding $\boldsymbol{W}$ to calculate future data.

Utilize pre-trained spatial feature matrix **W** of size [12, r], pre-trained data type feature matrix **V** of size [6, r], pre-trained LSTM coefficients **var(f)**, prestep temporal feature matrix **X0** of size [max(time_lags), r] and observations(may be incomplete) to make predictions of the next time step.

### 1. Current temporal embedding $x_t$ calibration
At time slot $t$, the newly observed data matrix $\boldsymbol{Y}_t\in\mathbb{R}^{M\times N}$ comes in, online LSTM-ReTF framework calibrates the previously predicted temporal feature vector $\boldsymbol{x}'_t$. The calibrated temporal feature vector $x_t$ can be derived from the following optimization problem:
$$
\min_{\boldsymbol{x}_t}~~\sum_{(i,j)\in\Omega_t}\left(y_{ij,t}-\sum_{r=1}^{R}u_{ir}v_{jr}x_{tr}\right)^2+\frac{\lambda_{x}\eta}{2}\sum_{t=1}^{T}\left\|\boldsymbol{x}_{t}\right\|_{2}^{2}+\underbrace{\frac{\lambda_x}{2}\sum_{t=l_d+1}^n\biggl(\boldsymbol{x}_t - \boldsymbol{x}'_t\biggr)^2}_{\text{LSTM network regularizer}}
$$
where $\Omega_{t}$ denotes the observed entries in the real time observation matrix $\boldsymbol{Y}_{t}$.

This optimizatoin problem can be solved by least square method, the uodating formulation for temporal embeddding $x_t$ can be derived as follows:
$$
\boldsymbol{x}_{t} = \biggl(\sum_{(i,j)\in\Omega_{t}}(\boldsymbol{w}_i\circledast \boldsymbol{v}_j)(\boldsymbol{w}_i\circledast \boldsymbol{v}_j)^\top + \lambda_x I+\lambda_x\eta I \biggr)^{-1}\biggl(\sum_{(i,j)\in\Omega_{t}}y_{ij,t}(\boldsymbol{w}_i\circledast \boldsymbol{v}_j) + \lambda_x \boldsymbol{x}'_{t} \biggr)
$$

### 2. Missing data imputation in current observation matrix $\boldsymbol{Y}_{t}$
With calibrated temporal feature vector $\boldsymbol{x}_{t}$ and the pre-trained spatial feature matrix $\boldsymbol{W}$, pre-trained data type feature matrix $\boldsymbol{V}$, we can make imputation of the current observation by:
$$
{\hat{\boldsymbol{Y}_t}}_{i,j} = \sum_{k=1}^r \boldsymbol{w}_{ik}\boldsymbol{v}_{jk}\boldsymbol{x}'_{t, k}, i = 1, 2, ..., M, \ j= 1, 2, ..., N.
$$
where $\hat{\boldsymbol{Y}_t}\in\mathbb{R}^{M\times N}$ is the approximated observation matrix.

### 3. Future temporal embedding $\boldsymbol{x}_{t+1}$ prediction
Making forecast of future temporal feature vector $\boldsymbol{x}_{t+1}$.
$$
\boldsymbol{x}'_{t + 1} = f(\boldsymbol{x}_{t + 1 - l_1},\boldsymbol{x}_{t + 1 - l_2},...,\boldsymbol{x}_{t + 1 -l_d})
$$

### 4. Future data predction
With dynamically calculated temporal feature vector $\boldsymbol{x}_{t+1}$ and the pre-trained spatial feature matrix $\boldsymbol{W}$, pre-trained data type feature matrix $\boldsymbol{V}$, we can make prediction of the spatial temporal data of the next time slots as follows:
$$
{{\boldsymbol{Y}'_t}^{(t+1)}}_{i,j} = \sum_{k=1}^r \boldsymbol{w}_{ik}\boldsymbol{v}_{jk}\boldsymbol{x+1}'_{t, k}, i = 1, 2, ..., M, \ j= 1, 2, ..., N.
$$
where ${\boldsymbol{Y}'_{t+1}}\in\mathbb{R}^{M\times N}$ is the predicted observation matrix of the next time step.


### Online temporal embedding calibration algorithm

In [37]:
def OnlineLSTMReTF(sparse_mat, init, lambda_x, time_lags):
    time_lags = time_lags[::-1]
    W = init["W"]
    V = init["V"]
    X = init["X"]
    model = init["model"]
    dim1, dim2 = sparse_mat.shape
    t, rank = X.shape
    X_hat = X[t - 1 - time_lags, :].copy()
    X_hat_feed = X_hat[np.newaxis, :, :]
    Qt =  model.predict(X_hat_feed)[0]
    
    sparse_tensor = np.zeros((dim1, dim2, 1))
    sparse_tensor[:, :, 0] = sparse_mat
    position = np.where(sparse_tensor != 0)
    binary_tensor = np.zeros(sparse_tensor.shape)
    binary_tensor[position] = 1
    var1 = kr_prod(V, W).T
    var2 = kr_prod(var1, var1)
    var_mu = np.matmul(var1, ten2mat(sparse_tensor, 2).reshape([dim1 * dim2])) + lambda_x * Qt
    inv_var_Lambda = inv(np.matmul(var2, ten2mat(binary_tensor, 2).reshape([dim1 * dim2])).reshape([rank, rank]) + lambda_x * np.eye(rank))
    return np.matmul(inv_var_Lambda, var_mu)

### Online prediction framework

In [38]:
def online_prediction(sparse_tensor, init, time_lags, lambda_x, maxiter):
    W = init["W"]
    V = init["V"]
    X = init["X"]
    model = init["model"]
    pre_step_num = X.shape[0]
    rank = X.shape[1]
    dim1, dim2, dim3 = sparse_tensor.shape
    X_hat = np.zeros((dim3 + pre_step_num, rank))
    tensor_pred = np.zeros((dim1, dim2, dim3))
    X_hat[:pre_step_num,:] = X.copy()
    start_time = time.time()
    for t in range(dim3):
        if t == 0:
            X_star = X_hat[pre_step_num + t - time_lags, :][::-1]
            X_star_feed = X_star[np.newaxis, :, :]
            Qt =  model.predict(X_star_feed)[0]
            X_hat[pre_step_num + t, :] = Qt.copy()
        else:
            sparse_mat = sparse_tensor[:, :, t - 1]
            if np.where(sparse_mat > 0)[0].shape[0] > 0:
                init = {"W": W, "V": V, "X": X_hat[pre_step_num + t - np.max(time_lags) - 1 : pre_step_num + t, :],
                        "model": model}
                X_c = OnlineLSTMReTF(sparse_mat, init, lambda_x/dim3, time_lags)
                X_hat[pre_step_num + t - 1, :] = X_c.copy()
                X_star = X_hat[pre_step_num + t - time_lags, :][::-1]
                X_star_feed = X_star[np.newaxis, :, :]
                Qt =  model.predict(X_star_feed)[0]
                X_hat[pre_step_num + t, :] = Qt.copy()
            else:
                X_star = X_hat[pre_step_num + t - time_lags, :][::-1]
                X_star_feed = X_star[np.newaxis, :, :]
                Qt =  model.predict(X_star_feed)[0]
                X_hat[pre_step_num + t, :] = Qt.copy()
        tensor_pred[:, :, t] = np.einsum('ir, jr, r -> ij', W, V, X_hat[pre_step_num + t, :])
        if (t + 1) % 1000 == 0:
            print('Time step: %d, time cost: %d s'%((t + 1), (time.time() - start_time)))
            start_time = time.time()
            
    sparse_mat = sparse_tensor[:, :, -1]
    init = {"W": W, "V": V, "X": X_hat[dim2 + pre_step_num - np.max(time_lags) - 1 : , :], "model": model}
    X_c = OnlineLSTMReTF(sparse_mat, init, lambda_x/dim2, time_lags)
    X_hat[dim2 + pre_step_num - 1,:] = X_c.copy()
    tensor_rec = cp_combine(W, V, X_hat[pre_step_num : , :])
    return tensor_rec, tensor_pred

### Making prediction on test set

In [89]:
import time
start_time = time.time()
sc = 100000
init = {"W": W, "V": V, "X": X[- np.max(time_lags): , :], "model": model}
test_mat_rec, test_mat_pred = online_prediction(test_set, init, time_lags, sc * lambda_x, maxiter)
print('Shape of imputed data is:')
print(test_mat_rec.shape)
print()
print('Shape of predicted data is:')
print(test_mat_pred.shape)

Time step: 1000, time cost: 3 s
Time step: 2000, time cost: 3 s
Shape of imputed data is:
(170, 3, 2880)

Shape of predicted data is:
(170, 3, 2880)


In [90]:
print('10 first prediciton on test set:')
print(test_mat_pred[0, 0, :10])
print()
print('10 first real value on test set')
print(test_ground_truth[0, 0, :10])

10 first prediciton on test set:
[131.70135841 132.21074799 131.32163445 133.13654547 130.9236932
 128.13524087 124.30082013 118.62154246 111.75715259 112.41522463]

10 first real value on test set
[108. 107. 115. 110. 107. 108.  94.  92.  92.  93.]


### Calculate the prediction and imputation error
RMSE(root mean squared error) and MAPE(mean absolute percentage error) for both prediction and imputation on test set are calculated in the following code:

In [91]:
# Prediction
pos = np.where(test_ground_truth != 0)
testPred_rmse = root_mean_squared_error(test_ground_truth, test_mat_pred, pos)
print('Test prediction RMSE: %.2f RMSE' % (testPred_rmse))
testPred_mape = mean_absolute_percentage_error(test_ground_truth, test_mat_pred, pos)
print('Test prediction MAPE: %.2f%% MAPE' % (testPred_mape))
print()

# Imputation
pos = np.where((test_set == 0) & (test_ground_truth != 0))
testPred_rmse = root_mean_squared_error(test_ground_truth, test_mat_rec, pos)
print('Test prediction RMSE: %.2f RMSE' % (testPred_rmse))
testPred_mape = mean_absolute_percentage_error(test_ground_truth, test_mat_rec, pos)
print('Test prediction MAPE: %.2f%% MAPE' % (testPred_mape))

Test prediction RMSE: 17.48 RMSE
Test prediction MAPE: 20.73% MAPE

Test prediction RMSE: 18.15 RMSE
Test prediction MAPE: 21.34% MAPE


# License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>